In [1]:
import json
import itertools
from fractions import Fraction
import numpy as np
import pandas

#GUI elements
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

In [2]:
%run logic.ipynb
%run participant.ipynb


----------------------------------------------------------------------
Ran 0 tests in 0.000s

OK


In [3]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [4]:
%%html
<style>
.jupyter-widgets.widget-tab > .p-TabBar .p-TabBar-tab {
    flex: 0 1 180px
}
</style>


In [5]:
#Variable initialization

current_dist_index = 0

#noise_removal_filter = NoiseRemovalFilter.NoiseRemovalFilter()

#Default values
population_size = 100 # Participant per experiment
initial_budget = 10.
number_experiments = 10 # Times to run the experiment
json_name = 'blood.json'

# Parameters for output graphs
output_folder = 'output/'
output_format = 'eps'
label_font_size=8
xlabel_rotation = 10
ylimit=[0,50]

In [6]:
json_files = filter(lambda x: x.lower().endswith('.json'), [f for f in os.listdir('.')])

file_selection = widgets.Select(
    options=json_files,
    value=json_name,
    disabled=False
)

precision_text = widgets.BoundedIntText(value=2,
    disabled=False,
    step = 1,
    min=0)

epsilon_text = widgets.BoundedFloatText(
    value=initial_budget,
    disabled=False,
    readout=True,
    readout_format='.'+str(precision_text.value)+'f',
    step = 10**-precision_text.value,
    min = 10**-precision_text.value
)

number_experiments_text = widgets.BoundedIntText(
    value=number_experiments,
    disabled=False,
    step = 1,
    min=1)

output_folder_text = widgets.Text(
    value=output_folder,
    placeholder='Folder for output',
    disabled=False
)

output_format_selection = widgets.Select(
    options=['eps', 'png', 'pdf', 'svg'],
    value=output_format,
    disabled=False
)

load_button = widgets.Button(
    description='Load JSON',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    icon='hourglass-start'
)

dist_accordion = widgets.Accordion()
dist_accordion.layout.padding = '0% 0% 5% 1%'
dist_accordion.layout.width = '99%'

json_box = widgets.VBox([widgets.Label('Input JSON:'),file_selection, load_button])
json_box.layout.align_items='center'
exp_box = widgets.VBox([widgets.Label('Number of Experiments'),number_experiments_text])
exp_box.layout.align_items='center'
epsilon_box = widgets.VBox([widgets.Label('Total Privacy Budget:'), epsilon_text])
epsilon_box.layout.align_items='center'

params_box = widgets.HBox([json_box, exp_box, epsilon_box])
params_box.layout.align_items='flex-start'

precison_box = widgets.VBox([widgets.Label('Precision:'), precision_text])
precison_box.layout.align_items='center'
folder_box = widgets.VBox([widgets.Label('Output Folder:'), output_folder_text])
folder_box.layout.align_items='center'
format_box = widgets.VBox([widgets.Label('Output Format:'), output_format_selection])
format_box.layout.align_items='center'

output_box = widgets.HBox([precison_box, folder_box, format_box])
output_box.layout.align_items='flex-start'

start_tab = widgets.Tab()
tab_contents = ['Experiment Parameters', 'Output Options']
children = [params_box, output_box]
start_tab.children = children
for i in range(len(tab_contents)):
    start_tab.set_title(i, tab_contents[i])
    
run_button = widgets.Button(
    description='Run Experiments',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    icon='check'
)

run_box = widgets.VBox([run_button])
run_box.layout.width='99%'
run_box.layout.align_items='center'


number_html = widgets.HTML()

toggle_button = widgets.ToggleButton(
    description='Switch Mode',
    disabled=False,
    value=False,
    button_style='warning', # 'success', 'info', 'warning', 'danger' or ''
    icon='edit'
)   

toggle_box = widgets.VBox([toggle_button])
toggle_box.layout.width = '99%'
toggle_box.layout.align_items = 'center'

In [7]:
# GUI elements
uniform_buttons = []
draw_buttons = []
shape_sliders = []

sliders_precision = 4

dists = ['Custom', 'Geom', 'Logser', 'Planck', 'Uniform', 'Zipf']
dist_maps = []
dropdowns = []

sliders_matrix = []
custom_sliders = []
texts = []

dist_boxes = []

out = widgets.Output()
out.layout.align_items = 'center'
fig = None

In [8]:
#Example JSON
root_question = {'qid':0,
'question': 'Blood type?',
'answers': ['O+','A+','Other'],
'probability': ['1/3','1/3','1/3']}

follow_up_1 = {'qid':1,
'question': 'Specify...',
'answers': ['B+','AB+','O-','A-','Other'],
'probability': ['1/5','1/5','1/5','1/5','1/5']}

follow_up_2 = {'qid':2,
'question': 'Specify further...',
'answers': ['B-','AB-'],
'probability': ['1/2','1/2']}

paths = [[0,'Other',1], [1,'Other',2]]

json_poll = {
    'roots': [root_question],
    'children': [follow_up_1, follow_up_2],
    'paths':paths,
    'truth':['1/4']
}
json_poll

{'children': [{'answers': ['B+', 'AB+', 'O-', 'A-', 'Other'],
   'probability': ['1/5', '1/5', '1/5', '1/5', '1/5'],
   'qid': 1,
   'question': 'Specify...'},
  {'answers': ['B-', 'AB-'],
   'probability': ['1/2', '1/2'],
   'qid': 2,
   'question': 'Specify further...'}],
 'paths': [[0, 'Other', 1], [1, 'Other', 2]],
 'roots': [{'answers': ['O+', 'A+', 'Other'],
   'probability': ['1/3', '1/3', '1/3'],
   'qid': 0,
   'question': 'Blood type?'}],
 'truth': ['1/4']}

In [9]:
# Distribution sliders per question (non-flattened)
children = []

subtrees = pollToSubtrees(json_poll)
for question in subtrees:
    for subquestion in subtrees[question]:
        html = widgets.HTML('<b>Question:</b> '+subquestion['question'])
        to_add = []
        
        count = 0
        for answer in subquestion['answers']:
            probability = subquestion['probability'][count]
            to_add.append(widgets.Label(value=answer))
            to_add.append(widgets.FloatSlider(
                value=probability,
                min=0,
                max=1.0,
                step=0.01,
                description='Weight:',
                disabled=False,
                continuous_update=False,
                orientation='vertical',
                readout=True,
                readout_format='.1f',
            ))
            count +=1
        
        contents =[html, widgets.HBox(to_add)]
        children.append( widgets.VBox(contents))
        
dist_accordion.children = children
dist_box = widgets.VBox([dist_accordion])

In [10]:
display(start_tab)

VGFiKGNoaWxkcmVuPShIQm94KGNoaWxkcmVuPShWQm94KGNoaWxkcmVuPShMYWJlbCh2YWx1ZT11J0lucHV0IEpTT046JyksIFNlbGVjdChvcHRpb25zPSgnYmxvb2QuanNvbicsKSwgdmFsdWXigKY=


In [11]:
display(widgets.HTML('<center><h1> Input Distribution</h1></center>'))
display(dist_accordion)

HTML(value=u'<center><h1> Input Distribution</h1></center>')

QWNjb3JkaW9uKGNoaWxkcmVuPShWQm94KGNoaWxkcmVuPShIVE1MKHZhbHVlPXUnPGI+UXVlc3Rpb246PC9iPiBCbG9vZCB0eXBlPycpLCBIQm94KGNoaWxkcmVuPShMYWJlbCh2YWx1ZT11J0/igKY=


In [12]:
participants = []
n = 100000
participant_budget = 40

matrices = pollToMatrix(json_poll)
epsilons = pollToEpsilon(json_poll)

#TODO: Loop x times for confidence

for i in range(0, n):
    new_participant = Participant(participant_budget, json_poll)
    new_participant.setMatrices(matrices)
    new_participant.setEpsilons(epsilons)
    participants.append(new_participant)
    


In [13]:
#TODO: read weights from dist_accordion

weights = pollToPathWeights(json_poll)
ps = pollToPaths(json_poll)

ans_dist_zip = {}
for question in ps.keys():
    paths = ps[question]
    weight = weights[question]
    zipped = []
    for path in paths:
         zipped.append((path, weight[pathToKey(path)]))
    ans_dist_zip[question] = zipped

#participant.createAnswer(ans_dist_zip)

In [14]:
#TODO: set answer dist

#TODO: run RR

#TODO: compare to ground truth

#TODO: display graphs